<a href="https://colab.research.google.com/github/TsundereCoding/TsundereCoding/blob/main/Environment_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install -q -U kaggle_environments

In [11]:
import numpy as np
import pandas as pd
import random
import datetime
from collections import defaultdict

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

Опишем поведение агента, всегда играющего "камень" - это значение 0

In [12]:
%%writefile rock_agent.py

#Example of the simple agent
#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return 0

Overwriting rock_agent.py


Попробуем теперь использовать информацию о прошлых действиях противника. Опишем агента, который производит то же самое действие, что и оппонент на прошлом ходу

In [13]:
%%writefile copy_opponent.py

#Example
def copy_opponent(observation, configuration):
    #in case we have information about opponent last move
    if observation.step > 0:
        return observation.lastOpponentAction
    #initial step
    else:
        return random.randrange(0, configuration.signs)

Overwriting copy_opponent.py


In [14]:
%%writefile random_agent.py
#Случайный агент выбирает по random [1]
def random_agent(observation, configuration):
    return random.randint(0, 2)

Overwriting random_agent.py


In [15]:
%%writefile cycle_agent.py
#Агент, который чередует камень, ножницы и бумагу по порядку [2]
def cycle_agent(observation, configuration):
    return observation.step % 3

Overwriting cycle_agent.py


In [16]:
%%writefile copy_agentsc.py
#Агент, который повторяет ход противника с предыдущего раунда но начинает с ножниц [3]
def copy_agentsc(observation, configuration):
    if observation.step == 0:
        return 3 # Начинаем с ножниц
    return observation.lastOpponentAction

Writing copy_agentsc.py


In [17]:
%%writefile counter_agent.py
#Агент, который выбирает то, что победило бы предыдущий ход противника [4]
def counter_agent(observation, configuration):
    if observation.step == 0:
        return 0  # Начинаем с камня
    return (observation.lastOpponentAction + 1) % 3

Writing counter_agent.py


In [18]:
%%writefile lose_agent.py
#Агент, который всегда выбирает то, что проиграло бы предыдущему ходу противника [5]
def lose_agent(observation, configuration):
    if observation.step == 0:
        return 0  # Начинаем с камня
    return (observation.lastOpponentAction + 2) % 3

Writing lose_agent.py


In [19]:
%%writefile love_paper_agent.py
#Агент, который выбирает бумагу чаще других (любитель денег) [6]
def love_paper_agent(observation, configuration):
    return 2 if random.random() < 0.7 else random.randint(0, 1)

Writing love_paper_agent.py


In [20]:
%%writefile scared_rock_agent.py
#Агент, который выбирает случайные ходы, но избегает камня [7]
def scared_rock_agent(observation, configuration):
    return random.choice([1, 2])  # Только ножницы или бумага

Writing scared_rock_agent.py


In [21]:
%%writefile all_in_agent.py
#Агент, который выбирает самый часто проигрывающий ход из прошлых игр [8]
def all_in_agent(observation, configuration):
    if observation.step == 0:
        return random.randint(0, 2)
    moves = [0, 0, 0]
    for i in range(observation.step):
        moves[observation['actionHistory'][i]] += 1
    return moves.index(min(moves))

Writing all_in_agent.py


In [22]:
%%writefile lao_tzu_agent.py
#Агент, меняет стратегию каждый 3ий ход [9]
def lao_tzu_agent(observation, configuration):
    if observation.step % 10 < 3:
        return (observation.step % 3)
    else:
        return random.randint(0, 2)

Writing lao_tzu_agent.py


In [23]:
%%writefile time_agent.py
#Агент использует текущее время для выбора хода [10]

def time_agent(observation, configuration):
    current_time = datetime.datetime.now()
    if current_time.minute % 5 == 0:
        return 1  # если минута делится на 5 — выбирает ножницы
    elif current_time.minute % 2 == 0:
        return 0  # если время четное выбирает камень
    else:
        return 2  # если нечетное — бумагу

Writing time_agent.py


In [24]:
%%writefile birthday_agent.py
#Агент использует цифры дня рождения для выбора хода [11]
BIRTHDAY = "23011993"

def birthday_agent(observation, configuration):
    step = observation.step
    digit = int(BIRTHDAY[step % len(BIRTHDAY)])
    if digit % 3 == 0:
        return 0  # Камень
    elif digit % 3 == 1:
        return 1  # Ножницы
    else:
        return 2  # Бумага

Writing birthday_agent.py


In [25]:
%%writefile italian_agent.py
#Агент использующий числа Фибоначчи для выбора хода [12]
def fibonacci(self,n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        a, b = 0, 1
        for _ in range(2, n + 1):
            a, b = b, a + b
        return b

def italian_agent(self,observation, configuration):
    step = observation.step
    # Получаем n-ое число Фибоначчи
    fib_number = fibonacci(step % 10)  # Ограничиваем длину для предотвращения больших чисел

    # Используем цифры числа Фибоначчи для выбора хода:  мы берем остаток от деления этого числа на 3, чтобы выбрать один из трех ходов
    if fib_number % 3 == 0:
        return 0  # Камень
    elif fib_number % 3 == 1:
        return 1  # Ножницы
    else:
        return 2  # Бумага

Writing italian_agent.py


Агенты с элементами машинного обучения в них я не уверен совсем.

In [26]:
%%writefile q_learning_agent.py

#Агент использует стратегию Q-learning для выбора оптимальных ходов на основе прошлых данных. [13]

q_table = np.zeros((3, 3))  # Инициализация Q-таблицы

def q_learning_agent(observation, configuration):
    global q_table
    if observation.step == 0:
        return random.randint(0, 2)  # Начинаем с случайного хода как обычно

    opponent_move = observation.lastOpponentAction
    my_move = random.randint(0, 2)

    # Обновляем таблицу на основе предыдущего результата
    reward = 1 if (my_move - opponent_move) % 3 == 1 else -1
    q_table[my_move, opponent_move] += 0.1 * (reward - q_table[my_move, opponent_move])

    # Выбираем действие с максимальным значением Q
    return np.argmax(q_table[:, opponent_move])

Writing q_learning_agent.py


#15 agents to evaluate!!! дает ошибку, не совсем понял как это сдавать но КОЛАБ предложил вариант ниже



```
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "copy_opponent.py","random_agent.py","cycle_agent.py","copy_agentsc.py","counter_agent.py",
     "lose_agent.py","love_paper_agent.py","scared_rock_agent.py","all_in_agent.py","lao_tzu_agent.py","time_agent.py",
     "birthday_agent.py","italian_agent.py","q_learning_agent.py"],
    configuration={"episodeSteps": 100} #number of episodes
)```



In [43]:
import itertools

# List of agent scripts
agents = ["rock_agent.py", "copy_opponent.py", "random_agent.py", "cycle_agent.py", "copy_agentsc.py", "counter_agent.py",
          "lose_agent.py", "love_paper_agent.py", "scared_rock_agent.py", "all_in_agent.py", "lao_tzu_agent.py", "time_agent.py",
          "birthday_agent.py", "italian_agent.py", "q_learning_agent.py"]

# Dictionary to store wins for each agent
win_count = {agent: 0 for agent in agents}

# Iterate through all pairs of agents
for agent1, agent2 in itertools.combinations(agents, 2):
    print(f"Evaluating {agent1} vs {agent2}")
    # Evaluate match between agent1 and agent2
    result = evaluate(
        "rps",
        [agent1, agent2],
        configuration={"episodeSteps": 100}
    )

    # Process the results
    total_reward_1 = sum([r[0] for r in result if r[0] is not None])  # Total score for agent1, handling None
    total_reward_2 = sum([r[1] if r[1] is not None else 0 for r in result])  # Total score for agent2, handling None

    # Determine the winner
    if total_reward_1 > total_reward_2:
        win_count[agent1] += 1
    elif total_reward_2 > total_reward_1:
        win_count[agent2] += 1

# Find the agent with the most wins
most_wins = max(win_count.values())
top_agents = [agent for agent, wins in win_count.items() if wins == most_wins]

# Display the results
print("\n--- Results ---")
for agent, wins in win_count.items():
    print(f"{agent}: {wins} wins")

print("\nTop Agent(s) with the most wins:")
for agent in top_agents:
    print(f"{agent}: {most_wins} wins")

Evaluating rock_agent.py vs copy_opponent.py
Evaluating rock_agent.py vs random_agent.py
Evaluating rock_agent.py vs cycle_agent.py
Evaluating rock_agent.py vs copy_agentsc.py
Evaluating rock_agent.py vs counter_agent.py
Evaluating rock_agent.py vs lose_agent.py
Evaluating rock_agent.py vs love_paper_agent.py
Evaluating rock_agent.py vs scared_rock_agent.py
Evaluating rock_agent.py vs all_in_agent.py
Evaluating rock_agent.py vs lao_tzu_agent.py
Evaluating rock_agent.py vs time_agent.py
Evaluating rock_agent.py vs birthday_agent.py
Evaluating rock_agent.py vs italian_agent.py
Evaluating rock_agent.py vs q_learning_agent.py
Evaluating copy_opponent.py vs random_agent.py
Evaluating copy_opponent.py vs cycle_agent.py
Evaluating copy_opponent.py vs copy_agentsc.py
Evaluating copy_opponent.py vs counter_agent.py
Evaluating copy_opponent.py vs lose_agent.py
Evaluating copy_opponent.py vs love_paper_agent.py
Evaluating copy_opponent.py vs scared_rock_agent.py
Evaluating copy_opponent.py vs all